In [1]:
#from google.colab import files
#uploaded = files.upload()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('kidney_disease.csv')
df

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,bu,sc,sod,pot,hemo,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,36.0,1.2,NaN,NaN,15.4,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,18.0,0.8,NaN,NaN,11.3,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,53.0,1.8,NaN,NaN,9.6,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,56.0,3.8,111.0,2.5,11.2,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,26.0,1.4,NaN,NaN,11.6,35,7300,4.6,no,no,no,good,no,no,ckd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,395,55.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,140.0,49.0,0.5,150.0,4.9,15.7,47,6700,4.9,no,no,no,good,no,no,notckd
396,396,42.0,70.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,75.0,31.0,1.2,141.0,3.5,16.5,54,7800,6.2,no,no,no,good,no,no,notckd
397,397,12.0,80.0,1.020,0.0,0.0,normal,normal,notpresent,notpresent,100.0,26.0,0.6,137.0,4.4,15.8,49,6600,5.4,no,no,no,good,no,no,notckd
398,398,17.0,60.0,1.025,0.0,0.0,normal,normal,notpresent,notpresent,114.0,50.0,1.0,135.0,4.9,14.2,51,7200,5.9,no,no,no,good,no,no,notckd


Feature Details
Attribute Information:

Id(numerical) - Patient Id Age(numerical) - age in years

Blood Pressure(numerical) - bp in mm/Hg

Specific Gravity(nominal) - sg - (1.005,1.010,1.015,1.020,1.025)

Albumin(nominal) - al - (0,1,2,3,4,5)

Sugar(nominal) - su - (0,1,2,3,4,5)

Red Blood Cells(nominal) - rbc - (normal,abnormal)

Pus Cell (nominal) - pc - (normal,abnormal)

Pus Cell clumps(nominal) - pcc - (present,notpresent)

Bacteria(nominal) - ba - (present,notpresent)

Blood Glucose Random(numerical) - bgr in mgs/dl

Blood Urea(numerical) -bu in mgs/dl

Serum Creatinine(numerical) - sc in mgs/dl

Sodium(numerical) - sod in mEq/L

Potassium(numerical) - pot in mEq/L

Hemoglobin(numerical) - hemo in gms

Packed Cell Volume(numerical)

White Blood Cell Count(numerical) - wc in cells/cumm

Red Blood Cell Count(numerical) - rc in millions/cmm

Hypertension(nominal) - htn - (yes,no)

Diabetes Mellitus(nominal) - dm - (yes,no)

Coronary Artery Disease(nominal) - cad - (yes,no)

Appetite(nominal) - appet - (good,poor)

Pedal Edema(nominal) - pe - (yes,no)

Anemia(nominal) - ane - (yes,no)

Class (nominal)- class - (ckd,notckd)

In [ ]:
df.dtypes

id                  int64
age               float64
bp                float64
sg                float64
al                float64
su                float64
rbc                object
pc                 object
pcc                object
ba                 object
bgr               float64
bu                float64
sc                float64
sod               float64
pot               float64
hemo              float64
pcv                object
wc                 object
rc                 object
htn                object
dm                 object
cad                object
appet              object
pe                 object
ane                object
classification     object
dtype: object

In [ ]:
# EDA를 위한 데이터 분류
categoric_var=['sg','al','su','rbc','pc','pcc','ba','htn','dm','cad','appet','pe','ane']
numeric_var=['id','age','bp','bgr','bu','sc','sod','pot','hemo','pcv','wc','rc']

In [ ]:
# Categorical variables 들에서 잘못된 값 여부 체크
for col in categoric_var:
    print(f"unique() for {col} are \n {df[col].unique()}")

unique() for sg are 
 [1.02  1.01  1.005 1.015   nan 1.025]
unique() for al are 
 [ 1.  4.  2.  3.  0. nan  5.]
unique() for su are 
 [ 0.  3.  4.  1. nan  2.  5.]
unique() for rbc are 
 [nan 'normal' 'abnormal']
unique() for pc are 
 ['normal' 'abnormal' nan]
unique() for pcc are 
 ['notpresent' 'present' nan]
unique() for ba are 
 ['notpresent' 'present' nan]
unique() for htn are 
 ['yes' 'no' nan]
unique() for dm are 
 ['yes' 'no' ' yes' '\tno' '\tyes' nan]
unique() for cad are 
 ['no' 'yes' '\tno' nan]
unique() for appet are 
 ['good' 'poor' nan]
unique() for pe are 
 ['no' 'yes' nan]
unique() for ane are 
 ['no' 'yes' nan]


In [ ]:
# 'dm'과 'cad'에 있는 '\t' 처리
df['dm']=df.dm.replace("\tno","no")
df['dm']=df.dm.replace("\tyes","yes")

df['cad']=df.cad.replace("\tno","no")

In [ ]:
# numeric variables 들에서 잘못된 값 여부 체크
for col in numeric_var:
    print(f"unique() for {col} are \n {df[col].unique()}")

unique() for id are 
 [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 

In [ ]:
# 'pcv'의 잘못된 값 수정 및 숫자열로 변환
df['pcv'] = df['pcv'].replace(['\t?', '\t43'],[np.nan, '43'])
df['pcv'] = pd.to_numeric(df['pcv'])

In [ ]:
#'wc'와 'rc'에 대한 전처리 필요
# feature description에서 numeric variable이라 기술되어있는데, object로 보여지기 때문 
df[df['wc'].map(lambda x:type(x)==str)].wc.unique(), df[df['rc'].map(lambda x:type(x)==str)].rc.unique()

(array(['7800', '6000', '7500', '6700', '7300', '6900', '9600', '12100',
        '4500', '12200', '11000', '3800', '11400', '5300', '9200', '6200',
        '8300', '8400', '10300', '9800', '9100', '7900', '6400', '8600',
        '18900', '21600', '4300', '8500', '11300', '7200', '7700', '14600',
        '6300', '\t6200', '7100', '11800', '9400', '5500', '5800', '13200',
        '12500', '5600', '7000', '11900', '10400', '10700', '12700',
        '6800', '6500', '13600', '10200', '9000', '14900', '8200', '15200',
        '5000', '16300', '12400', '\t8400', '10500', '4200', '4700',
        '10900', '8100', '9500', '2200', '12800', '11200', '19100', '\t?',
        '12300', '16700', '2600', '26400', '8800', '7400', '4900', '8000',
        '12000', '15700', '4100', '5700', '11500', '5400', '10800', '9900',
        '5200', '5900', '9300', '9700', '5100', '6600'], dtype=object),
 array(['5.2', '3.9', '4.6', '4.4', '5', '4.0', '3.7', '3.8', '3.4', '2.6',
        '2.8', '4.3', '3.2', '3.6', '4'

In [ ]:
# 'wc'와 'rc'를 numeric variable로 변환하고, \t과 같은 특수문자가 들어간 data를 nan으로 변환
df['wc'] = pd.to_numeric(df['wc'],errors='coerce')
df['rc'] = pd.to_numeric(df['rc'],errors='coerce')